In [11]:
import pandas as pd

ooc = pd.read_csv("/home/hduva/projects/xrr_notebooks/fitting/optical_constants.csv")

In [13]:
import pyref.fitting as fit
import numpy as np

ZNPC = "C32H16N8Zn"
MA = np.asin(np.sqrt(2 / 3))


def setp_from_tensor(slab, tensor):
    """Set parameters from tensor."""
    rbounds = (
        tensor.real.min() - 1e-3,
        tensor.real.max() + 1e-3,
    )
    ibounds = (
        0.0002,
        tensor.imag.max() + 5e-3,
    )
    slab.sld.xx.setp(vary=True, bounds=rbounds)
    slab.sld.zz.setp(vary=True, bounds=rbounds)
    slab.sld.ixx.setp(vary=True, bounds=ibounds)
    slab.sld.izz.setp(vary=True, bounds=ibounds)
    return slab


def vacuum(energy):
    """Vacuum."""
    slab = fit.MaterialSLD("", 0, name=f"Vacuum_{energy}")(0, 0)
    slab.thick.setp(vary=False)
    slab.rough.setp(vary=False)
    slab.sld.density.setp(vary=False)
    return slab


def substrate(energy, rough=1.5, density=2.3):
    """Substrate."""
    slab = fit.MaterialSLD(
        "Si", density=density, energy=energy, name=f"Substrate_{energy}"
    )(0, rough)
    slab.thick.setp(vary=False)
    slab.rough.setp(vary=False)
    slab.sld.density.setp(vary=True, bounds=(2, 3))
    return slab


def sio2(energy, thick=13, rough=3, density=2.28):
    """SiO2."""
    slab = fit.MaterialSLD(
        "SiO2", density=density, energy=energy, name=f"Oxide_{energy}"
    )(thick, rough)
    slab.thick.setp(vary=True, bounds=(1, 15))
    slab.rough.setp(vary=True, bounds=(0, 15))
    slab.sld.density.setp(vary=True, bounds=(1, 2.5))
    return slab


def contamination(energy, thick=12, rough=7, density=1.0):
    """Contamination."""
    name = f"Contamination_{energy}"
    if energy < 270:
        slab = fit.MaterialSLD(ZNPC, density=density, energy=energy, name=name)(
            thick, rough
        )
        slab.sld.density.setp(vary=True, bounds=(1, 2))
    else:
        iso = fit.UniTensorSLD(ooc, density=density, rotation=0, energy=energy).tensor
        slab = fit.SLD(iso, name=name)(thick, rough)
        slab = setp_from_tensor(slab, iso)

    slab.thick.setp(vary=True, bounds=(1, 18))
    slab.rough.setp(vary=True, bounds=(0, 18))
    return slab


def surface(energy, thick=11, rough=7, density=1.0):
    """Surface."""
    name = f"Surface_{energy}"
    if energy < 270:
        slab = fit.MaterialSLD(ZNPC, density=density, energy=energy, name=name)(
            thick, rough
        )
        slab.sld.density.setp(vary=True, bounds=(1, 1.8))
    else:
        iso = fit.UniTensorSLD(ooc, density=density, rotation=0, energy=energy).tensor
        slab = fit.SLD(iso, name=name)(thick, rough)
        slab = setp_from_tensor(slab, iso)

    slab.thick.setp(vary=True, bounds=(1, 20))
    slab.rough.setp(vary=True, bounds=(0, 10))
    return slab


def znpc(energy, thick=170, rough=1.5, density=1.614):
    """ZnPc."""
    name = f"ZnPc_{energy}"
    if energy < 270:
        slab = fit.MaterialSLD(ZNPC, density=density, energy=energy, name=name)(
            thick, rough
        )
        slab.sld.density.setp(vary=True, bounds=(1.2, 1.8))
    if energy > 270 and energy <= 284.2:
        slab = fit.UniTensorSLD(
            ooc, density=density, rotation=1.35, energy=energy, name=name
        )(thick, rough)
        slab.sld.density.setp(vary=True, bounds=(1.2, 1.8))
        slab.sld.rotation.setp(vary=True, bounds=(MA, np.pi / 2))
    if energy > 284:
        iso = fit.UniTensorSLD(
            ooc, density=density, rotation=np.pi / 2, energy=energy
        ).tensor
        slab = fit.SLD(iso, name=name)(thick, rough)
        slab = setp_from_tensor(slab, iso)

    slab.thick.setp(vary=True, bounds=(150, 210))
    slab.rough.setp(vary=True, bounds=(0, 18))
    return slab


def construct_slab(
    energy,
    thickness=[10, 170, 10, 10],
    roughness=[7, 1.5, 7, 3, 1],
    density=[1.0, 1.614, 1.6, 2.28, 2.3],
    offset=0,
):
    """Construct the slab."""
    offset_energy = round(energy + offset, 1)
    slab = (
        vacuum(offset_energy)
        | surface(
            offset_energy, thick=thickness[0], rough=roughness[0], density=density[0]
        )
        | znpc(
            offset_energy, thick=thickness[1], rough=roughness[1], density=density[1]
        )
        | contamination(
            offset_energy, thick=thickness[2], rough=roughness[2], density=density[2]
        )
        | sio2(
            offset_energy, thick=thickness[3], rough=roughness[3], density=density[3]
        )
        | substrate(offset_energy, rough=roughness[4], density=density[4])
    )
    return slab


def construct_model(
    energy,
    thetas,
    thickness=[10, 170, 10, 10],
    roughness=[7, 1.5, 7, 3, 1],
    density=[1.0, 1.614, 1.6, 2.28, 2.3],
    scale_s=1.0,
    scale_p=1.0,
    theta_offset_s=0.0,
    theta_offset_p=0.0,
    bkg=0,
):
    """Construct the model."""
    slab = construct_slab(energy, thickness, roughness, density)
    models = fit.ReflectModel(
        slab,
        energy=energy,
        pol="s",
        scale_s=scale_s,
        scale_p=scale_p,
        theta_offset_s=theta_offset_s,
        theta_offset_p=theta_offset_p,
        bkg=bkg,
    )
    modelp = fit.ReflectModel(
        slab,
        energy=energy,
        pol="p",
        scale_s=scale_s,
        scale_p=scale_p,
        theta_offset_s=theta_offset_s,
        theta_offset_p=theta_offset_p,
        bkg=bkg,
    )
    #  Calculate q from theta values
    wavelength = 12398.42 / energy
    q = 4 * np.pi / wavelength * np.sin(np.radians(thetas))
    s = models(q)
    p = modelp(q)
    return s, p


#  Define the parameter space that we will use for generating the training data

#  =============/ Energies/ =========================
# Energies will be from 280 to 320 eV, with a step size of 0.1 eV
energies = np.arange(280, 320, 0.1)

# =============/ Thetas/ =========================
# Theta will be from 1 degree to 5, 10, 25, 60 degrees
# The step sizes will be every 0.1 degrees, and every .5 degrees
thetas = np.concatenate(
    [
        np.arange(1, 5, 0.1),
        np.arange(1, 5, 0.5),
        np.arange(1, 10, 0.1),
        np.arange(1, 10, 0.5),
        np.arange(1, 25, 0.1),
        np.arange(1, 25, 0.5),
        np.arange(1, 60, 0.1),
        np.arange(1, 60, 0.5),
    ]
)

# ============/ Thicknesses/ =========================
# Thicknesses will be from 0 to 200 nm, with a step size of 10 nm
thicknesses0 = np.arange(0, 200, 10)  # now clone for each layer
thicknesses1 = thicknesses0.copy()
thicknesses2 = thicknesses0.copy()
thicknesses3 = thicknesses0.copy()
# ============/ Roughnesses/ =========================
# Roughnesses will be from 0 to 20 nm, with a step size of 1 nm
roughnesses0 = np.arange(0, 20, 1)  # now clone for each layer
roughnesses1 = roughnesses0.copy()
roughnesses2 = roughnesses0.copy()
roughnesses3 = roughnesses0.copy()
roughnesses4 = roughnesses0.copy()  # for substrate
# ============/ Densities/ =========================
# Densities will be from 0.5 to 3 g/cm^3, with a step size of 0.1 g/cm^3
densities0 = np.arange(0.5, 3, 0.1)  # clone for each layer
densities1 = densities0.copy()
densities2 = densities0.copy()
densities3 = densities0.copy()
densities4 = densities0.copy()
# ============/ Scale factors/ =========================
# Scale factors will be from 0.1 to 2, with a step size of 0.01
scale_factors = np.arange(
    0.1, 2, 0.01
)  # constant for all layers and is just a simple scale factor
# ============/ Theta offsets/ =========================
# Theta offsets will be from -0.1 to 0.1 degrees, with a step size of 0.01 degrees
theta_offsets = np.arange(-0.1, 0.1, 0.01)
#  Use constant background of 0

In [16]:
from concurrent.futures import ThreadPoolExecutor
import random

energies = np.arange(280, 320, 0.5)  # Reduced step size
thetas = np.arange(1, 60, 0.1)

# Use sampling instead of full grid
n_samples = 100000  # Adjust based on your needs


# Generate random parameter combinations
def generate_random_params(n_samples):
    params = []
    for _ in range(n_samples):
        param = {
            "energy": random.choice(energies),
            "theta": thetas,  # constant for all samples
            "thickness": [random.choice(thicknesses0) for _ in range(4)],
            "roughness": [random.choice(roughnesses0) for _ in range(5)],
            "density": [random.choice(densities0) for _ in range(5)],
            "scale_s": random.choice(scale_factors),
            "scale_p": random.choice(scale_factors),
            "theta_offset_s": random.choice(theta_offsets),
            "theta_offset_p": random.choice(theta_offsets),
        }
        params.append(param)
    return params


# Function to evaluate model for a single parameter set
def evaluate_model(params):
    s, p = construct_model(
        params["energy"],
        params["theta"],
        thickness=params["thickness"],
        roughness=params["roughness"],
        density=params["density"],
        scale_s=params["scale_s"],
        scale_p=params["scale_p"],
        theta_offset_s=params["theta_offset_s"],
        theta_offset_p=params["theta_offset_p"],
    )
    return s, p


# Generate parameter combinations
param_combinations = generate_random_params(n_samples)

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(evaluate_model, param_combinations))

# Separate s and p data
s_data = [result[0] for result in results]
p_data = [result[1] for result in results]

# Convert to numpy arrays
s_data = np.array(s_data)
p_data = np.array(p_data)
